# How how to use `get_rm_from_altaz`

You can also, for a given time, map the ionospheric rotation measure sky above the observer
by giving a set of azimuth and elevation coordinates

In [ ]:
from __future__ import annotations

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import AltAz, EarthLocation
from astropy.time import Time
from astropy.visualization import quantity_support, time_support
from spinifex import get_rm

In [ ]:
### Required to load local data for example - not needed for normal use
from importlib import resources

with resources.as_file(resources.files("spinifex.data.tests")) as datapath:
    spinifex_data = datapath
###

In [ ]:
times = Time("2020-01-08T01:00:00")
# create AltAz grid
azangles = np.linspace(0, 360, 30)
altangles = np.linspace(10, 90, 8)
altazangles = np.meshgrid(azangles, altangles)
az = altazangles[0] * u.deg
alt = altazangles[1] * u.deg
# create Earth Location
lon = 6.367 * u.deg
lat = 52.833 * u.deg
dwingeloo = EarthLocation(lon=lon, lat=lat, height=0 * u.km)
# make altaz object, including location and times
# get_rm expects a flattened array of coordinates, you can reshape the returned results
altaz = AltAz(az=az.flatten(), alt=alt.flatten(), location=dwingeloo, obstime=times)
# get rotation measures
rm = get_rm.get_rm_from_altaz(
    loc=dwingeloo,
    altaz=altaz,
    # We set these options to use the data packaged with spinifex
    # Unsetting them will cause the function to download the data from the internet
    prefix="cod",
    output_directory=spinifex_data,
)
rotation_measures = rm.rm.reshape(az.shape)
rm_times = rm.times

In [ ]:
with time_support(), quantity_support():
    fig, ax = plt.subplots()
    im = ax.pcolormesh(
        az,
        alt,
        rotation_measures,
        cmap=plt.cm.RdBu_r,
        vmin=-2,
        vmax=2,
        shading="auto",
    )
    ax.set_xlabel("Azimuth")
    ax.set_ylabel("Altitude")

    fig.colorbar(im, label="Rotation Measure [rad/m$^2$]")
    fig, ax = plt.subplots(subplot_kw={"projection": "polar"})
    im = ax.pcolormesh(
        az.to(u.rad).value,
        90 - alt.to(u.deg).value,
        rotation_measures,
        cmap=plt.cm.RdBu_r,
        vmin=-2,
        vmax=2,
        shading="auto",
    )
    fig.colorbar(im, label="Rotation Measure [rad/m$^2$]")